In [1]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

### 1. 讀入深度學習套件

In [12]:
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding
from tensorflow.keras.layers import LSTM
from tensorflow.keras.datasets import imdb

### 2.讀入數據

In [6]:
(x_train, y_train), (x_test, y_test)= imdb.load_data(num_words=10000)

In [21]:
len(x_train)

25000

In [22]:
len(x_test)

25000

In [18]:
x_train[0]

[1,
 14,
 22,
 16,
 43,
 530,
 973,
 1622,
 1385,
 65,
 458,
 4468,
 66,
 3941,
 4,
 173,
 36,
 256,
 5,
 25,
 100,
 43,
 838,
 112,
 50,
 670,
 2,
 9,
 35,
 480,
 284,
 5,
 150,
 4,
 172,
 112,
 167,
 2,
 336,
 385,
 39,
 4,
 172,
 4536,
 1111,
 17,
 546,
 38,
 13,
 447,
 4,
 192,
 50,
 16,
 6,
 147,
 2025,
 19,
 14,
 22,
 4,
 1920,
 4613,
 469,
 4,
 22,
 71,
 87,
 12,
 16,
 43,
 530,
 38,
 76,
 15,
 13,
 1247,
 4,
 22,
 17,
 515,
 17,
 12,
 16,
 626,
 18,
 2,
 5,
 62,
 386,
 12,
 8,
 316,
 8,
 106,
 5,
 4,
 2223,
 5244,
 16,
 480,
 66,
 3785,
 33,
 4,
 130,
 12,
 16,
 38,
 619,
 5,
 25,
 124,
 51,
 36,
 135,
 48,
 25,
 1415,
 33,
 6,
 22,
 12,
 215,
 28,
 77,
 52,
 5,
 14,
 407,
 16,
 82,
 2,
 8,
 4,
 107,
 117,
 5952,
 15,
 256,
 4,
 2,
 7,
 3766,
 5,
 723,
 36,
 71,
 43,
 530,
 476,
 26,
 400,
 317,
 46,
 7,
 4,
 2,
 1029,
 13,
 104,
 88,
 4,
 381,
 15,
 297,
 98,
 32,
 2071,
 56,
 26,
 141,
 6,
 194,
 7486,
 18,
 4,
 226,
 22,
 21,
 134,
 476,
 26,
 480,
 5,
 144,
 30,
 5535,
 18,

In [12]:
len(x_train[0])

218

In [13]:
len(x_train[2])

141

In [14]:
y_train[0]

1

In [15]:
y_train[1]

0

### 3. 資料處理

In [39]:
x_train = sequence.pad_sequences(x_train, maxlen=100)
x_test = sequence.pad_sequences(x_test, maxlen= 100)

### 4. step 01 : 打造一個函數學習機

In [76]:
model = Sequential()

In [77]:
model.add(Embedding(10000, 128))

In [78]:
model.add(LSTM(128, dropout=0.2, recurrent_dropout=0.2))

In [45]:
model.add(Dense(1, activation = 'sigmoid'))

In [47]:
model.compile(loss='binary_crossentropy',
             optimizer='adam',
             metrics=['accuracy'])

In [49]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (None, None, 128)         1280000   
_________________________________________________________________
lstm_3 (LSTM)                (None, 128)               131584    
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 129       
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 2         
Total params: 1,411,715
Trainable params: 1,411,715
Non-trainable params: 0
_________________________________________________________________


In [51]:
(128+128+1)*4*128

131584

### 5. step 02 :訓練

In [52]:
model.fit(x_train, y_train, batch_size=32, epochs=10,
         validation_data= (x_test, y_test))

Train on 25000 samples, validate on 25000 samples
Epoch 1/10
25000/25000 [==============================] - 257s 10ms/sample - loss: 0.6746 - acc: 0.5713 - val_loss: 0.6657 - val_acc: 0.5798
Epoch 2/10
25000/25000 [==============================] - 268s 11ms/sample - loss: 0.6255 - acc: 0.6874 - val_loss: 0.7070 - val_acc: 0.5949
Epoch 3/10
25000/25000 [==============================] - 299s 12ms/sample - loss: 0.6597 - acc: 0.6198 - val_loss: 0.5773 - val_acc: 0.7675
Epoch 4/10
25000/25000 [==============================] - 306s 12ms/sample - loss: 0.5486 - acc: 0.7816 - val_loss: 0.5214 - val_acc: 0.7979
Epoch 5/10
25000/25000 [==============================] - 310s 12ms/sample - loss: 0.4714 - acc: 0.8376 - val_loss: 0.5021 - val_acc: 0.7974
Epoch 6/10
25000/25000 [==============================] - 329s 13ms/sample - loss: 0.4125 - acc: 0.8675 - val_loss: 0.4549 - val_acc: 0.8192
Epoch 7/10
25000/25000 [==============================] - 320s 13ms/sample - loss: 0.3609 - acc: 0.8887 

In [54]:
model_json = model.to_json()
open('imdb_model_architecture.json','w').write(model_json)
model.save_weights('imdb_model_weights.h5')

## 根據老師的步驟，可以看出準確率跟正確性都表現得蠻好的，但花了快30分鐘才跑完，所以我接下來的改變希望可以加快速度，並且觀察學習的正確性是否會受影響。

#### 1.改變輸入資料長度

In [7]:
x_train = sequence.pad_sequences(x_train, maxlen=75)
x_test = sequence.pad_sequences(x_test, maxlen= 75)

#### 2.降低 dropout,epochs

In [15]:
model = Sequential()

In [16]:
model.add(Embedding(10000, 128))

In [17]:
model.add(LSTM(128, dropout=0.1, recurrent_dropout=0.1))

In [18]:
model.add(Dense(1, activation = 'sigmoid'))

In [21]:
model.compile(loss='binary_crossentropy',
             optimizer='adam',
             metrics=['accuracy'])

In [20]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, None, 128)         1280000   
_________________________________________________________________
lstm_2 (LSTM)                (None, 128)               131584    
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 129       
Total params: 1,411,713
Trainable params: 1,411,713
Non-trainable params: 0
_________________________________________________________________


In [22]:
model.fit(x_train, y_train, batch_size=32, epochs=5,
         validation_data= (x_test, y_test))

Train on 25000 samples, validate on 25000 samples
Epoch 1/5
25000/25000 [==============================] - 238s 10ms/sample - loss: 0.4536 - acc: 0.7851 - val_loss: 0.3853 - val_acc: 0.8317
Epoch 2/5
25000/25000 [==============================] - 265s 11ms/sample - loss: 0.2996 - acc: 0.8776 - val_loss: 0.3745 - val_acc: 0.8369
Epoch 3/5
25000/25000 [==============================] - 284s 11ms/sample - loss: 0.2279 - acc: 0.9090 - val_loss: 0.4131 - val_acc: 0.8318
Epoch 4/5
25000/25000 [==============================] - 297s 12ms/sample - loss: 0.1692 - acc: 0.9348 - val_loss: 0.5085 - val_acc: 0.8279
Epoch 5/5
25000/25000 [==============================] - 308s 12ms/sample - loss: 0.1263 - acc: 0.9534 - val_loss: 0.5729 - val_acc: 0.8181


#### 結論:每個EPOCH跑的時間並沒有明顯減少，但因為減少了EPOCH數量減半，等待時間明顯降低嘞，而減少輸入資料的長度跟DROPOUT 根據數據表現來看，並沒有降低正確性。